# Ising spins on a ladder

$$
-\beta\mathcal{H} = K\sum^2_{n=1}\sum^M_{m=1}(S_{m,n}S_{m+1,n} + S_{m,n}S_{m,n+1})
$$
with periodic boundary condition

In [1]:
using Random
using StatsBase

TList = collect(range(start=0.1, stop=5.0, length=50))
const MCstep = 10000
const K = 1.0

const length = 4    # length of ladder
const N = length * 2 # number of site on ladder

println("")

## Initiate Lattice

In [83]:
function InitLattice(N)
    #Lattice = ones(Int8, N) # all site is up
    #Lattice = -1 * ones(Int8, N) # all site is down
    #Lattice = sample([-1, 1], Weights([1, 1]), N) # all site random
    Lattice = collect(1:N) # for debug
    
    return Lattice
end

InitLattice (generic function with 2 methods)

In [84]:
function PeriodicBoundaryCondition(row, col)
    # All site has 3 of degrees
    adj = zeros(Int8, (row*col, 3))
    
    for i = 1:row
        for j = 1:col
            idx = (i-1)*row + j
            #Horizontal
            adj[idx, 1] = (i-1)*col + ((j + col) % col + 1)
            adj[idx, 2] = (i-1)*col + ((j + col - 2) % col + 1)
            #Vertical
            if i==1
                tmp3 = col + j
                adj[idx, 3] = col + j
            end
            if i==2
                tmp3 = j
                adj[idx, 3] = j
            end
        end
    end                
    return adj
end             

PeriodicBoundaryCondition (generic function with 1 method)

In [85]:
function calcE(Lattice, row, col, i, j)
    E = 0
    E += Lattice[i][j] * Lattice[i][(col + j + 1) % col + 1] #East
    E += Lattice[i][j] * Lattice[i][(col + j - 1) % col + 1] #West
    E += Lattice[i][j] * Lattice[(row + i + 1) % row + 1][j] #South
    E += Lattice[i][j] * Lattice[(row + i - 1) % row + 1][j] #North
    return E
end

calcE (generic function with 1 method)

In [86]:
function FlipSpin(Lattice, row, col)
    i, j = rand(1:length), rand(1:2)
    E = calcE(Lattice, length, 2, i, j)
end

FlipSpin (generic function with 1 method)

In [87]:
Lattice = InitLattice(N)
adj = PeriodicBoundaryCondition(2, length)
#E = calcE(Lattice, 2, length, 1, 1)
"""
for T in TList
    for step 1:MCstep
        mag = 0.0
        FlipSpin(Lattice, length, mag)

    end
end
"""

"for T in TList\n    for step 1:MCstep\n        mag = 0.0\n        FlipSpin(Lattice, length, mag)\n\n    end\nend\n"